# Import libraries

In [1]:
# numpy, scipy, pandas, sklearn, matplotlib
import numpy as np
import pandas as pd
from scipy.stats import entropy
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score,RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

# pytorch and pytorch lightning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision 
from torchvision import datasets
import torchvision.transforms as transforms
!pip install torchsummary
from torchsummary import summary
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping

# others
import os
from tqdm import tqdm_notebook as tqdm
import time
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)
# print("Number of workers:", NUM_WORKERS)

Device: cuda:0


In [3]:
# global constants
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 3
BATCH_SIZE = 32

# Import data

In [4]:
# define image transformation when importing data
def images_transforms_original(training = False):
    if training:
        data_transformation = transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            # transforms.RandomEqualize(10),
            transforms.RandomRotation(degrees = 15),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
    else:
        data_transformation = transforms.Compose([
            transforms.Resize(IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        
    
    return data_transformation

In [5]:
# dataset path
train_path = '../input/covid19-image-dataset/Covid19-dataset/train'
val_path = '../input/covid19-image-dataset/Covid19-dataset/test/'
deploy_path = '../input/covid19radiographydatabaseedited/COVID-19_Radiography_Dataset'

# import data
trainset_original = datasets.ImageFolder(train_path,transform=images_transforms_original(training = True))
valset_original = datasets.ImageFolder(val_path,transform=images_transforms_original())
deployset_original = datasets.ImageFolder(deploy_path,transform=images_transforms_original())

# reduce size of deployment data (for computational time and memory)
n_deployment = 800
deploy_idx = np.random.choice(len(deployset_original.targets), size=n_deployment, replace=False)
deployset_original = Subset(deployset_original, deploy_idx)

print("Training set size: {}\nValidation set size: {}\nDeployment set size: {}".format(len(trainset_original.targets),len(valset_original.targets),n_deployment))
print(trainset_original.class_to_idx)


Training set size: 251
Validation set size: 66
Deployment set size: 800
{'Covid': 0, 'Normal': 1, 'Viral Pneumonia': 2}


# Original model

In [6]:
class Original(pl.LightningModule):
    def __init__(self, lr, weight_decay, num_classes = NUM_CLASSES, pretrained_model = None, max_epochs=10):
        super().__init__()
        self.save_hyperparameters()
        self.model = pretrained_model if pretrained_model is not None else torchvision.models.resnet50(pretrained=True)  # Output of last linear layer: 2048-dim representation
        if pretrained_model is None:
            self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr,
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                            T_max=self.hparams.max_epochs,
                                                            eta_min=self.hparams.lr/50) # search what does it refer to
        return [optimizer], [lr_scheduler]

    def crossentropy_loss(self, batch, mode='train'):
        images, labels = batch
        outputs = self.model(images)
        loss = F.cross_entropy(outputs, labels)
        self.log(mode+'_loss', loss)
        return loss

    def training_step(self, batch, batch_idx):
        return self.crossentropy_loss(batch, mode='train')
    
    def validation_step(self, batch, batch_idx):
        self.crossentropy_loss(batch, mode='val')

In [7]:
def train_original(train_loader, val_loader, batch_size=32, pretrained_model=None, max_epochs=20, **kwargs):
    trainer = pl.Trainer(gpus=1 if str(device)=='cuda:0' else 0,
                         max_epochs=max_epochs,
                         callbacks=[pl.callbacks.EarlyStopping(monitor="val_loss")])
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    pl.seed_everything(42) # To be reproducable
    model = Original(pretrained_model=pretrained_model, max_epochs=max_epochs, **kwargs)
    trainer.fit(model, train_loader, val_loader)
    model = Original.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    return model

In [8]:
# hyperparameters
LR = 1e-3
WD = 1e-4

# DataLoader
train_loader_original = DataLoader(trainset_original,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
val_loader_original = DataLoader(valset_original,batch_size=BATCH_SIZE,shuffle=False,num_workers=4)
deploy_loader_original = DataLoader(deployset_original,batch_size=BATCH_SIZE,shuffle=False,num_workers=4)

original_model = train_original(train_loader=train_loader_original,
                            val_loader=val_loader_original,
                            batch_size=BATCH_SIZE,
                            lr=LR,
                            weight_decay=WD)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [9]:
def test(model, testloader):
    with torch.no_grad():
        n_correct=0
        n_samples=0
        y_pred=[]
        y_actual=[]
        for i,(images,labels) in enumerate(testloader):
            outputs=model(images)
            
            y_actual+=list(np.array(labels.detach().to('cpu')).flatten())
        # value ,index
            _,predictes=torch.max(outputs,1)
            y_pred+=list(np.array(predictes.detach().to('cpu')).flatten())
        # number of samples in current batch
            n_samples+=labels.shape[0]

            n_correct+= (predictes==labels).sum().item()
            
        y_actual=np.array(y_actual).flatten()
        y_pred=np.array(y_pred).flatten()
        print("Unique classes predicted: {}".format(np.unique(y_pred)))
        acc = classification_report(y_actual,y_pred,target_names=valset_original.classes)
        print(f"{acc}")

In [10]:
trainset_norotate = datasets.ImageFolder(train_path,transform=images_transforms_original())
train_loader_norotate = DataLoader(trainset_norotate,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
test(original_model, train_loader_norotate)

Unique classes predicted: [0 1 2]
                 precision    recall  f1-score   support

          Covid       1.00      1.00      1.00       111
         Normal       1.00      0.96      0.98        70
Viral Pneumonia       0.96      1.00      0.98        70

       accuracy                           0.99       251
      macro avg       0.99      0.99      0.99       251
   weighted avg       0.99      0.99      0.99       251



In [11]:
test(original_model, val_loader_original)

Unique classes predicted: [0 1 2]
                 precision    recall  f1-score   support

          Covid       0.61      0.73      0.67        26
         Normal       0.90      0.95      0.93        20
Viral Pneumonia       0.50      0.35      0.41        20

       accuracy                           0.68        66
      macro avg       0.67      0.68      0.67        66
   weighted avg       0.67      0.68      0.67        66



In [12]:
test(original_model, deploy_loader_original)

Unique classes predicted: [0 1 2]
                 precision    recall  f1-score   support

          Covid       0.33      0.76      0.46       194
         Normal       0.88      0.28      0.42       536
Viral Pneumonia       0.34      0.91      0.50        70

       accuracy                           0.45       800
      macro avg       0.52      0.65      0.46       800
   weighted avg       0.70      0.45      0.44       800



# Selection of new data for labeling

In [13]:
y_pred_prob=[]
deploy_images = torch.empty((0,3,)+IMAGE_SIZE)
deploy_labels = torch.empty(0)
for i, (images, labels) in enumerate(tqdm(deploy_loader_original, total=int(len(deploy_loader_original)))):
    deploy_images = torch.cat((deploy_images, images), axis=0)
    deploy_labels = torch.cat((deploy_labels, labels))
    # NOTE: the above only stores the images and labels for the selection, the algorithm for the decision of selection
    # does not make use of the labels
    outputs = original_model.model(images)
    outputs_prob = nn.Softmax(dim = 1)(outputs)
    y_pred_prob += list(outputs_prob.detach().cpu().numpy())
    
y_pred_prob = np.array(y_pred_prob)
y_pred_entropy = entropy(y_pred_prob, base=2, axis=1)
print(y_pred_entropy.shape)

  0%|          | 0/25 [00:00<?, ?it/s]

(800,)


In [14]:
print(deploy_images.shape)
print(deploy_labels.shape)

torch.Size([800, 3, 224, 224])
torch.Size([800])


# Random Selection and Model Adjustment

In [15]:
class CustomDataset(Dataset):
    # Convert torch.Tensor data and labels into a Dataset object
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels.type(torch.LongTensor)
    def __len__(self):
        return self.labels.size(dim=0)
    def __getitem__(self, idx):
        img = self.images[idx,:,:,:]
        class_id = self.labels[idx]
        return img, class_id

In [16]:
newset_all = CustomDataset(deploy_images, deploy_labels)
print(newset_all.__len__())

800


In [17]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets, transforms=None):
        self.datasets = datasets
        self.transforms = transforms

    def __getitem__(self, i):
        # print(self.__len__())
        try:
            ds_position = 0
            item = None
            while ds_position < len(self.datasets):
                if i < self.datasets[ds_position].__len__():
                    if self.transforms is not None:
                        item = (self.transforms(self.datasets[ds_position].__getitem__(i)[0]),
                           int(self.datasets[ds_position].__getitem__(i)[1]))
                    else:
                        item = (self.datasets[ds_position].__getitem__(i)[0], int(self.datasets[ds_position].__getitem__(i)[1]))
                    break
                else:
                    i -= self.datasets[ds_position].__len__()
                    ds_position += 1
            assert item is not None
            return item
        except:
            print("The function failed to get the item.")

    def __len__(self):
        return sum([self.datasets[i].__len__() for i in range(len(self.datasets))])

In [18]:
class SimCLR(pl.LightningModule):

    def __init__(self, hidden_dim, lr, temperature, weight_decay, pretrained_model=None, max_epochs=500):
        super().__init__()
        self.save_hyperparameters()
        assert self.hparams.temperature > 0.0, 'The temperature must be a positive float!'
        # Base model f(.)
        self.model = pretrained_model if pretrained_model is not None else torchvision.models.resnet50(pretrained=True)  # Output of last linear layer: 2048-dim representation
        # print("Hi\n{}".format(list(self.model.children())))
        # The MLP for g(.) consists of Linear->ReLU->Linear
        # print(self.model.fc)
        # print(len(list(list(self.model.children())[-1].children())))
        # print(list(self.model.children())[-1])
        # print(list(list(self.model.children())[-1].children()))
        if len(list(list(self.model.children())[-1].children())) == 0:
            self.model = nn.Sequential(*(list(self.model.children())[:-1]),
                                        nn.Flatten(),
                                        nn.Linear(self.model.fc.in_features, 2048),
                                        nn.ReLU(inplace=True),
                                        nn.Linear(2048, hidden_dim))
            # print("done")
        # print("Hihi\n{}".format(list(self.model.children())))
    
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),
                                lr=self.hparams.lr,
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                            T_max=self.hparams.max_epochs,
                                                            eta_min=self.hparams.lr/50) # search what does it refer to
        return [optimizer], [lr_scheduler]

    def info_nce_loss(self, batch, mode='train'):
        imgs, _ = batch
        # print(imgs.shape)
        imgs = torch.cat(imgs, dim=0)
        # print(imgs.shape)

        # Encode all images
        feats = self(imgs)
        # print(feats.shape)
        # Calculate cosine similarity
        cos_sim = F.cosine_similarity(feats[:,None,:], feats[None,:,:], dim=-1)
        # Mask out cosine similarity to itself
        self_mask = torch.eye(cos_sim.shape[0], dtype=torch.bool, device=cos_sim.device)
        cos_sim.masked_fill_(self_mask, -9e15)
        # Find positive example -> batch_size//2 away from the original example
        pos_mask = self_mask.roll(shifts=cos_sim.shape[0]//2, dims=0)
        # InfoNCE loss
        cos_sim = cos_sim / self.hparams.temperature
        nll = -cos_sim[pos_mask] + torch.logsumexp(cos_sim, dim=-1)
        nll = nll.mean()

        # Logging loss
        print(mode+'_loss: {:.4f}'.format(nll))
        self.log(mode+'_loss', nll)
        # Get ranking position of positive example
        comb_sim = torch.cat([cos_sim[pos_mask][:,None],  # First position positive example
                              cos_sim.masked_fill(pos_mask, -9e15)],
                             dim=-1)
        sim_argsort = comb_sim.argsort(dim=-1, descending=True).argmin(dim=-1)
        # Logging ranking metrics
        self.log(mode+'_acc_top1', (sim_argsort == 0).float().mean())
        self.log(mode+'_acc_top5', (sim_argsort < 5).float().mean())
        self.log(mode+'_acc_mean_pos', 1+sim_argsort.float().mean())

        return nll

    def training_step(self, batch, batch_idx):
        return self.info_nce_loss(batch, mode='train')

    def validation_step(self, batch, batch_idx):
        self.info_nce_loss(batch, mode='val')

In [19]:
simclr_model = SimCLR.load_from_checkpoint("../input/ssrchestxray/ssr-chest-x-ray.ckpt")
new_model = nn.Sequential(*(list(simclr_model.model.children())[:-3]), list(original_model.model.children())[-1])
# print(new_model)
# print(original_model)

In [20]:
# hyperparameters
LR = 1e-4
WD = 1e-4

# adjustment training
adjustset_train_ratio = 0.8
trainset_adjust = datasets.ImageFolder(train_path,transform=images_transforms_original())
valset_adjust = datasets.ImageFolder(val_path,transform=images_transforms_original())

SELECTION_PERCENTAGES = [5,10,25,100]
for p in SELECTION_PERCENTAGES:
    print("Selection percentage = {}%".format(p))
    indices = np.random.choice(deploy_labels.shape[0], size=int(deploy_labels.shape[0]*p/100), replace=False)
    deploy_images_selected, deploy_labels_selected = deploy_images[indices,:,:,:], deploy_labels[indices]
    print("Deploy images:\nCovid count: {}\nNormal count: {}\nViral pneumonia count: {}\n".format(sum(deploy_labels==0.), sum(deploy_labels==1.), sum(deploy_labels==2.)))
    newset_selected = CustomDataset(deploy_images_selected, deploy_labels_selected)
    if p != 100:
        adjustset = ConcatDataset(trainset_adjust, valset_adjust, newset_selected, transforms=None)
    else:
        adjustset = ConcatDataset(trainset_adjust, valset_adjust, newset_all, transforms=None)
    
    train_idx = np.random.choice(adjustset.__len__(), size=int(adjustset.__len__()*adjustset_train_ratio), replace=False)
    adjust_train_Trans = transforms.RandomRotation(degrees=15)
    adjustset_train = Subset(adjustset, train_idx)
    adjustset_train = ConcatDataset(adjustset_train, transforms=adjust_train_Trans)
    adjustset_val = Subset(adjustset, [i for i in range(adjustset.__len__()) if i not in train_idx])
    adjust_train_loader = DataLoader(adjustset_train,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
    adjust_val_loader = DataLoader(adjustset_val,batch_size=BATCH_SIZE,shuffle=False,num_workers=4)
    
    new_model_iter = train_original(train_loader=adjust_train_loader,
                            val_loader=adjust_val_loader,
                            batch_size=BATCH_SIZE,
                            lr=LR,
                            weight_decay=WD,
                            pretrained_model = new_model)
    print("Training data")
    test(new_model_iter, train_loader_norotate)
    print("Validation data")
    test(new_model, val_loader_original)
    print("Deployment data")
    test(new_model, deploy_loader_original)
    

Selection percentage = 5%
Deploy images:
Covid count: 194
Normal count: 536
Viral pneumonia count: 70



Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Training data
Unique classes predicted: [0 1 2]
                 precision    recall  f1-score   support

          Covid       1.00      0.71      0.83       111
         Normal       0.63      1.00      0.77        70
Viral Pneumonia       1.00      0.87      0.93        70

       accuracy                           0.84       251
      macro avg       0.88      0.86      0.85       251
   weighted avg       0.90      0.84      0.84       251

Validation data
Unique classes predicted: [0 1 2]
                 precision    recall  f1-score   support

          Covid       0.65      0.58      0.61        26
         Normal       0.51      1.00      0.68        20
Viral Pneumonia       1.00      0.20      0.33        20

       accuracy                           0.59        66
      macro avg       0.72      0.59      0.54        66
   weighted avg       0.72      0.59      0.55        66

Deployment data
Unique classes predicted: [0 1 2]
                 precision    recall  f1-score  